In [0]:
import os  
import requests  
import uuid  
import json  
!pip install azure-ai-translation-document==1.0.0
!pip install --upgrade pip
from azure.ai.translation.document import DocumentTranslationClient
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2


In [0]:
# Set the subscription key and endpoint for the Azure Translator Text API  
subscription_key = "c2324a985a1741a18f47c83ac8b506c4" 
endpoint = 'https://trrsltrlntr.cognitiveservices.azure.com/' 

In [0]:
# Define the language codes for the source and target languages  
source_language = 'en'  

In [0]:
# Ask the user to enter the list of target languages to translate the input text to  
target_languages_str = input("Enter the list of target languages to translate the input text to, separated by commas (e.g., de,fr,nl): ")  
target_languages = target_languages_str.split(',') 

Enter the list of target languages to translate the input text to, separated by commas (e.g., de,fr,nl):  de,nl,fr

In [0]:
target_languages

['de', 'nl', 'fr']

In [0]:
# Define the connection string and container name for the Azure Blob Storage account  
connection_string = 'DefaultEndpointsProtocol=https;AccountName=flklntransln;AccountKey=+lZTIj8Np3Xjx+tl85LjTwOit7ABoOjeXai94pwvpihMi82lHypNcwngDs9ehJAGxpkXPZCTaVbN+AStz/9IhA==;EndpointSuffix=core.windows.net'  
container_name = 'wrdflklntrlnsource'

In [0]:
# Define the name of the input file in Blob Storage  
input_file_name = 'input.txt' 

In [0]:
# Create a BlobServiceClient object to access the Blob Storage account  
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [0]:
# Get a BlobClient object for the input file  
blob_client = blob_service_client.get_blob_client(container=container_name, blob=input_file_name)

In [0]:
# Download the input file contents  
input_text = blob_client.download_blob().content_as_text()

In [0]:
# Define the translation headers and payload  
headers = {  
    'Ocp-Apim-Subscription-Key': subscription_key,  
    'Content-type': 'application/json',  
    'X-ClientTraceId': str(uuid.uuid4())  
}  
  
body = [{  
    'text': input_text  
}]

In [0]:
translation_url = f'{endpoint}/translate?api-version=3.0&from={source_language}&to={target_languages}'
print(translation_url)
response = requests.post(translation_url, headers=headers, json=body)  
response_data = json.loads(response.content.decode('utf-8'))  
print(response)

https://trrsltrlntr.cognitiveservices.azure.com//translate?api-version=3.0&from=en&to=['de', 'nl', 'fr']
<Response [404]>


In [0]:
  
    # Extract the translated text from the response  
    output_text = response_data
    output_text

{'error': {'code': '404', 'message': 'Resource not found'}}

In [0]:
# Translate the input text to each target language and save the output to a file  
for target_language in target_languages:  
    # Define the translation URL and payload  
    translation_url = f'{endpoint}/translate?api-version=3.0&from={source_language}&to={target_languages}' 
    print(translation_url)
    response = requests.post(translation_url, headers=headers, json=body)  
    #print(response)
    response_data = json.loads(response.content.decode('utf-8'))  
    

https://trrsltrlntr.cognitiveservices.azure.com//translate?api-version=3.0&from=en&to=['de', 'nl', 'fr']
https://trrsltrlntr.cognitiveservices.azure.com//translate?api-version=3.0&from=en&to=['de', 'nl', 'fr']
https://trrsltrlntr.cognitiveservices.azure.com//translate?api-version=3.0&from=en&to=['de', 'nl', 'fr']


In [0]:
    
    # Extract the translated text from the response  
    output_text = response_data[0]['translations'][0]['text']  

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File <command-2646505977091294>, line 2
      1 # Extract the translated text from the response  
----> 2 output_text = response_data[0]['translations'][0]['text']  

KeyError: 0

In [0]:
# Save the translated output to a file in Blob Storage  
    output_file_name = f'output_{target_languages}.txt'  
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=output_file_name)  
    blob_client.upload_blob(output_text, overwrite=True)